In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import *
import numpy as np
from tqdm import tqdm

from attention_graph_util import *
from notebooks.notebook_utils import *
from util import inflect

from tqdm import tqdm
import math
import pandas as pd

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

[nltk_data] Downloading package punkt to /home/dehghani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

In [3]:
data = tfds.load('glue/sst2')

# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='sst-2')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='sst-2')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
valid_dataset = valid_dataset.batch(64)
test_dataset = glue_convert_examples_to_features(data['test'], tokenizer, max_length=128, task='sst-2')
test_dataset = test_dataset.batch(1)

# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model.evaluate(valid_dataset, steps=100)
# Train and evaluate using tf.keras.Model.fit()
# history = model.fit(train_dataset, epochs=2, steps_per_epoch=115,
#                     validation_data=valid_dataset, validation_steps=7)

INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Reusing dataset glue (/home/dehghani/tensorflow_datasets/glue/sst2/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split None, from /home/dehghani/tensorflow_datasets/glue/sst2/1.0.0


 14/100 [===>..........................] - ETA: 15s - loss: 0.3824 - accuracy: 0.9106WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 100 batches). You may need to use the repeat() function when building your dataset.


 14/100 [===>..........................] - ETA: 16s - loss: 0.3605 - accuracy: 0.9106

[0.05353218950331211, 0.9105505]

In [4]:
model.config.output_attentions = True
model.config.output_hidden_states = True

model.distilbert.transformer.output_attentions = True
model.distilbert.transformer.output_hidden_states = True

for layer in model.distilbert.transformer.layer:
    layer.output_attentions = True    
    layer.output_hidden_states = True
    layer.attention.output_attentions = True
    layer.attention.output_hidden_states = True
  
all_examples_grads = []
all_examples_attentions = []
all_examples_x = []
n_examples = 10
for x,y in test_dataset:
    with tf.GradientTape() as tape:
        inputs_embeds = model.distilbert.embeddings(x['input_ids'])
        tape.watch(inputs_embeds)
        logits, hidden_states, attentions = model({'attention_mask':x['attention_mask'], 
                                                    'inputs_embeds':inputs_embeds,
                                                    'token_type_ids': x['token_type_ids']}, training=False
                                            )
    grads = tape.gradient(logits, inputs_embeds)[0]  
    
    length = tf.reduce_sum(x['attention_mask'], axis=-1)[0]    
    all_examples_grads.append(tf.reduce_sum(grads, -1)[:length])
    
    _attentions = [att.numpy() for att in attentions]
    attentions_mat = np.asarray(_attentions)[:,0]
        
    cropped_input = x['input_ids'][0, :length]
    all_examples_x.append(cropped_input)
    cropped_attention_mat = attentions_mat[:,:,:length,:length]
    all_examples_attentions.append(cropped_attention_mat)
        
    if n_examples == 0:
        break
    n_examples -= 1

In [10]:
print("compute raw relevance scores ...")
all_examples_raw_relevance = {}
for l in np.arange(5,6):
    all_examples_raw_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokens = tokenizer.convert_ids_to_tokens(all_examples_x[i])
        length = len(tokens)
        attention_relevance = get_raw_att_relevance(all_examples_attentions[i], tokens, layer=l)
        all_examples_raw_relevance[l].append(np.asarray(attention_relevance))

print("compute joint relevance scores ...")
all_examples_joint_relevance = {}
for l in np.arange(5,6):
    all_examples_joint_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokenizer.convert_ids_to_tokens(all_examples_x[i])
        length = len(tokens)
        attention_relevance = get_joint_relevance(all_examples_attentions[i], tokens, layer=l)
        all_examples_joint_relevance[l].append(np.asarray(attention_relevance))
    
print("compute flow relevance scores ...")
all_examples_flow_relevance = {}
for l in np.arange(5,6):
    all_examples_flow_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokenizer.convert_ids_to_tokens(all_examples_x[i])
        length = len(tokens)            
        attention_relevance = get_flow_relevance(all_examples_attentions[i], tokens, layer=l)
        all_examples_flow_relevance[l].append(np.asarray(attention_relevance))

  0%|          | 0/11 [00:00<?, ?it/s]

compute raw relevance scores ...
compute joint relevance scores ...
compute flow relevance scores ...


100%|██████████| 11/11 [07:21<00:00, 40.14s/it]


In [11]:
for l in np.arange(5,6):
    print("###############Layer ",l, "#############")

    print('raw grad')
    print(all_examples_raw_relevance[l][0].shape, all_examples_grads[0].shape)
    raw_sps_grad = []
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_raw_relevance[l][i],all_examples_grads[i])
        raw_sps_grad.append(sp[0])
 
        
    print(np.mean(raw_sps_grad), np.std(raw_sps_grad))

    
    print('joint grad')
    print(all_examples_joint_relevance[l][0].shape, all_examples_grads[0].shape)
    joint_sps_grad = []
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_joint_relevance[l][i],all_examples_grads[i])
        joint_sps_grad.append(sp[0])

        
    print(np.mean(joint_sps_grad), np.std(joint_sps_grad))

  
    print('flow grad')
    print(all_examples_joint_relevance[l][0].shape, all_examples_grads[0].shape)
    flow_sps_grad = []
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_flow_relevance[l][i],all_examples_grads[i])
        flow_sps_grad.append(sp[0])
        
    print(np.mean(flow_sps_grad), np.std(flow_sps_grad))

###############Layer  5 #############
raw grad
(10,) (10,)
0.18592326895485767 0.22469418350775205
joint grad
(10,) (10,)
0.13513215271910164 0.24435241432073573
flow grad
(10,) (10,)
0.14945322598912117 0.27389137937753844


In [ ]:
np.mean([spearmanr(all_examples_flow_relevance[l][i], all_examples_joint_relevance[l][i]) for i in np.arange(len(all_examples_grads))])
np.mean([spearmanr(all_examples_flow_relevance[l][i], all_examples_grads[i]) for i in np.arange(len(all_examples_grads))])

In [12]:
all_examples_flow_relevance

{5: [array([1.24035012, 0.93157125, 0.97967586, 0.96986086, 0.94293693,
         1.00084612, 0.84883115, 0.91959955, 0.87702609, 1.28930207]),
  array([1.1434315 , 0.86609638, 0.9724092 , 0.97755364, 1.00281283,
         0.9805801 , 0.98228776, 0.94042922, 1.04347942, 1.1219926 ,
         1.10584035, 0.92329394, 0.99170254, 1.0021536 , 0.97752429,
         0.99547764, 0.94352372, 0.98432639, 1.05517316, 1.02422106,
         0.95763375, 0.91448882, 1.07936272, 1.03992586, 0.9505393 ,
         0.87531768, 1.00266451, 1.14575802]),
  array([1.14701559, 0.85198326, 0.91960435, 0.79398134, 0.98698866,
         0.9664473 , 0.8912048 , 1.02763052, 1.01868156, 1.0353566 ,
         1.00065476, 0.89913483, 1.02688355, 0.98035642, 1.09027183,
         0.98589747, 1.07790875, 1.06463144, 1.0773603 , 1.15800666]),
  array([1.15613104, 0.91679789, 0.87655517, 1.03186445, 0.97646101,
         1.09713204, 0.99981023, 1.00752057, 0.89128593, 0.92499432,
         1.001589  , 0.9944515 , 1.01012924, 0.96

In [9]:

def spearmanr(x, y):
    """ `x`, `y` --> pd.Series"""
    x = pd.Series(x)
    y = pd.Series(y)
    assert x.shape == y.shape
    rx = x.rank(method='dense')
    ry = y.rank(method='dense')
    d = rx - ry
    dsq = np.sum(np.square(d))
    n = x.shape[0]
    coef = 1. - (6. * dsq) / (n * (n**2 - 1.))
    return [coef]

def get_raw_att_relevance(full_att_mat, input_tokens, layer=-1):
    cls_index = 0
    return full_att_mat[layer].sum(axis=0).sum(axis=0)
    

def compute_node_flow(G, labels_to_index, input_nodes, output_nodes,length):
    number_of_nodes = len(labels_to_index)
    flow_values=np.zeros((number_of_nodes,number_of_nodes))
    for key in output_nodes:
        if key not in input_nodes:
            current_layer = int(labels_to_index[key] / length)
            pre_layer = current_layer - 1
            u = labels_to_index[key]
            for inp_node_key in input_nodes:
                v = labels_to_index[inp_node_key]
                flow_value = nx.maximum_flow_value(G,u,v)
                flow_values[u][pre_layer*length+v] = flow_value
            #normalize flow values
            flow_values[u] /= flow_values[u].sum()
            
    return flow_values

def get_flow_relevance(full_att_mat, input_tokens, layer):
    
    res_att_mat = full_att_mat.sum(axis=1)/full_att_mat.shape[1]
    res_att_mat = res_att_mat + np.eye(res_att_mat.shape[1])[None,...]
    res_att_mat = res_att_mat / res_att_mat.sum(axis=-1)[...,None]

    res_adj_mat, res_labels_to_index = get_adjmat(mat=res_att_mat, input_tokens=input_tokens)
    
    A = res_adj_mat
    res_G=nx.from_numpy_matrix(A, create_using=nx.DiGraph())
    for i in np.arange(A.shape[0]):
        for j in np.arange(A.shape[1]):
            nx.set_edge_attributes(res_G, {(i,j): A[i,j]}, 'capacity')

    output_nodes = []
    input_nodes = []
    for key in res_labels_to_index:
        if key.startswith('L'+str(layer+1)+'_'):
            output_nodes.append(key)
        if res_labels_to_index[key] < full_att_mat.shape[-1]:
            input_nodes.append(key)
    
    flow_values = compute_node_flow(res_G, res_labels_to_index, input_nodes, output_nodes, length=full_att_mat.shape[-1])
    
    n_layers = full_att_mat.shape[0]
    length = full_att_mat.shape[-1]
    final_layer_attention_raw = flow_values[(layer+1)*length: (layer+2)*length,layer*length: (layer+1)*length]
    relevance_attention_raw = final_layer_attention_raw.sum(axis=0)

    return relevance_attention_raw
    
    
def get_joint_relevance(full_att_mat, input_tokens, layer):
    att_sum_heads =  full_att_mat.sum(axis=1)/full_att_mat.shape[1]
    joint_attentions = compute_joint_attention(att_sum_heads, add_residual=True)
    relevance_attentions = joint_attentions[layer].sum(axis=0)
    return relevance_attentions


def get_nores_joint_relevance(full_att_mat, input_tokens, layer):
    att_sum_heads =  full_att_mat.sum(axis=1)/full_att_mat.shape[1]
    joint_attentions = compute_joint_attention(att_sum_heads, add_residual=False)
    relevance_attentions = joint_attentions[layer].sum(axis=0)
    return relevance_attentions